In [5]:
from Bio import pairwise2

from hypothesis import given
import hypothesis.strategies as st

# Computing longest common sub-sequences

Approach uses an implementation of the [Needleman-Wunsch Algorithm](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm) to compute the global alignment score and then the observation that this algorithm will reproduce the length of the longest common subsequence by setting the mismatch penatly, $\mu=0$, and the insertion-deletion penalty, $\sigma=0$.

We can then use a production implementation from [Biopython](https://biopython.org/) to confirm that our algorithm has been correctly implemented.

In [35]:
def needleman_wunsch_algorithm(s1, s2, mu, sigma):
    """Implementation of the Needleman-Wunsch Algorithm for computing the global alignment score."""
    m = len(s1)
    n = len(s2)

    D = _alignment_score_init(m, n, sigma)

    for i in range(m):
        for j in range(n):
            _alignment_score_step(D, i + 1, j + 1, s1, s2, mu, sigma)
    return D[m][n]


def _alignment_score_init(m, n, sigma):
    D = []
    for i in range(m + 1):
        D.append([0] * (n + 1))
    for i in range(m + 1):
        D[i][0] = sigma * i
    for j in range(n + 1):
        D[0][j] = sigma * j
    return D


def _alignment_score_step(D, i, j, s1, s2, mu, sigma):
    c1 = s1[i - 1]
    c2 = s2[j - 1]

    insertion = D[i][j - 1] + sigma
    deletion = D[i - 1][j] + sigma
    match = D[i - 1][j - 1] + 1
    mismatch = D[i - 1][j - 1] + mu

    if c1 == c2:
        D[i][j] = max(insertion, deletion, match)
    else:
        D[i][j] = max(insertion, deletion, mismatch)


In [37]:
@given(st.text(alphabet="ACTG", min_size=1, max_size=1000), st.text(alphabet="ACTG", min_size=1, max_size=1000))
def test_needleman_wunsch_algorithm(a, b):
    assert needleman_wunsch_algorithm(a, b, 0, 0) == pairwise2.align.globalxx(a, b, score_only=True)

In [38]:
test_needleman_wunsch_algorithm()